# AIxChem
### Visual exploration
#### Datasets — Dimensionality reduction

In [ ]:
from pathlib import Path
import matplotlib.colors

from aixchem import Dataset
from aixchem.transform import preprocess
from aixchem.models import decomposition
from aixchem.plots.scatter import Scatter
import matplotlib.pyplot as plt


AIxChem = Path.cwd().parent
DATA = AIxChem / "datasets" / "buttar_norrby_dataset.csv"
LABELS = "exp_activation_energy"

# Load the data & clean it
data = Dataset(DATA, target=LABELS, sep=",").dropna(axis=0).shuffle(random_state=42)
# Drop the index column
data.X.drop(columns=["Unnamed: 0"], inplace=True)
# Drop highly correlated features
data.correlation(thr=0.8)

scaled = preprocess.Scaler().fit(data).transform(data)

embeddings = {
    "pca": decomposition.PCA(n_components=2).run(scaled).embedding,
    "umap": decomposition.UMAP(n_components=2).run(scaled).embedding,
    "tsne": decomposition.tSNE(n_components=2).run(scaled).embedding,
}


#fig, ax = plt.subplots(3, 1, figsize=(4, 4*3))
fig, ax = plt.subplots(1, 3, figsize=(4*3, 4))
#cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#800040", "#DFE8F3","#004080"])
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#C8D4E3", "#004080"])
fig.tight_layout()

for idx, (embeddor, e) in enumerate(embeddings.items()):

    scatter = Scatter(
        x=e.X.iloc[:, 0], 
        y=e.X.iloc[:, 1], 
        colors=e.y[f"{LABELS}"],
        sizes=8,
        ax=ax[idx],
        colormap=cmap
        )

    scatter.set_axis_style(xlabel=e.X.iloc[:, 0].name, ylabel=e.X.iloc[:, 1].name)

# After creating all plots
fig.tight_layout()
plt.savefig("embeddings.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from pathlib import Path
import matplotlib.colors
import matplotlib.pyplot as plt

from aixchem import Dataset
from aixchem.transform import preprocess
from aixchem.models import decomposition
from aixchem.plots.scatter import Scatter


AIxChem = Path.cwd().parent
DATA = AIxChem / "datasets" / "buttar_norrby_dataset.csv"
LABELS = "exp_activation_energy"

# Load the data & clean it
data = Dataset(DATA, target=LABELS, sep=",").dropna(axis=0).shuffle(random_state=42)
# Drop the index column
data.X.drop(columns=["Unnamed: 0"], inplace=True)
# Drop highly correlated features
data.correlation(thr=0.8)


embeddings = {
    "pca": decomposition.PCA(n_components=2).run(scaled).embedding,
    "umap": decomposition.UMAP(n_components=2).run(scaled).embedding,
    "tsne": decomposition.tSNE(n_components=2).run(scaled).embedding,
}


#fig, ax = plt.subplots(3, 1, figsize=(4, 4*3))
fig, ax = plt.subplots(1, 3, figsize=(4*3, 4))
#cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#800040", "#DFE8F3","#004080"])
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#C8D4E3", "#004080"])
fig.tight_layout()

for idx, (embeddor, e) in enumerate(embeddings.items()):

    scatter = Scatter(
        x=e.X.iloc[:, 0], 
        y=e.X.iloc[:, 1], 
        colors=e.y[f"{LABELS}"],
        sizes=8,
        ax=ax[idx],
        colormap=cmap
        )

    scatter.set_axis_style(xlabel=e.X.iloc[:, 0].name, ylabel=e.X.iloc[:, 1].name)

# Perfomance Plot

Compare augmented and non-augmented performance using a performance plot

In [ ]:

from pathlib import Path
import pandas as pd

import matplotlib.pyplot as plt
from aixchem.plots.core import Plot

# Define the path to the results
AIxChem = Path.cwd().parent
RESULTS = AIxChem / "docs/buttar_norrby_results/holdout_RMSE"

# Define the metric to plot
METRIC = "holdout_RMSE"

color_map = {
    "AGN": "#1f77b4",
    "NNSMOTE": "#ff7f0e",
    "TVAE": "#2ca02c",
    "CTGAN": "#d62728",
}

# Read all csv files from the dir
files = [f for f in RESULTS.iterdir() if f.suffix == ".csv" and not "noaug" in f.stem and not "std" in f.stem and not "FOLD" in f.stem]

# Group results according to model
results = {model: [f for f in files if model in f.stem] for model in set([f.stem.split("_")[1] for f in files])}

# Handle both single and multiple subplot cases
n_models = len(results.keys())
fig, axes = plt.subplots(1, n_models, figsize=(4.5*n_models, 4), squeeze=False)
axes = axes.flatten()  # Flatten in case of single subplot


for idx, (model, result) in enumerate(results.items()):
    # Create model plot
    plot = Plot(ax=axes[idx])

    unaugmented_df = pd.read_csv(f"{result[0].parent / '_'.join(result[0].stem.split('_')[:2])}_noaug.csv", index_col=0)
    #unaugmented_df.drop(labels="5%", inplace=True)  # Drop the 5% row
    plot.ax.plot(unaugmented_df["FRAC"], unaugmented_df[METRIC], label="Baseline", color="#000000", linestyle="--")
    for r in result:
        df = pd.read_csv(r, index_col=0)
        # df.drop(labels="5%", inplace=True) # Drop the 5% row
        label = r.stem.split("_")[0]
        color = color_map.get(label, "#000000")  # Default to black if label not in color_map

        plot.ax.plot(df["FRAC"], df[METRIC], label=r.stem.split("_")[0], marker="s", markersize=3, color=color, alpha=0.5)
        plot.ax.set_title(model)
        # Show legend
        plot.ax.legend()
        # Set axis limits
        plot.ax.set_ylim(1, 3.5)        
        plot.ax.set_xticks(df["FRAC"])
        plot.ax.set_xticklabels(df.index)
        plot.ax.set_xlabel("Fraction of train data (75%)")
        plot.ax.set_ylabel(METRIC)


# Bar plot with standard deviation

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from aixchem.plots.core import Plot


# Define the path to the results and the metric to plot
AIxChem = Path.cwd().parent
RESULTS = AIxChem / "docs/buttar_norrby_results/holdout_RMSE"
METRIC = "holdout_RMSE"
SORT = "MAX" 
 
# Read all csv files from the dir
files = [f for f in RESULTS.iterdir() if f.suffix == ".csv" and not "noaug" in f.stem and not "std" in f.stem]

# Group results according to model
results = {model: [f for f in files if model in f.stem] for model in set([f.stem.split("_")[1] for f in files])}

# Handle both single and multiple subplot cases
n_models = len(results.keys())
fig, axes = plt.subplots(1, n_models, figsize=(4.5*n_models, 4), squeeze=False)
axes = axes.flatten()  # Flatten in case of single subplot

for idx, (model, result) in enumerate(results.items()):

    # Load non-augmented mean data
    unaugmented_df = pd.read_csv(f"{result[0].parent / '_'.join(result[0].stem.split('_')[:2])}_noaug.csv", index_col=0)
    
    # Load non-augmented standard deviation data
    try:
        unaugmented_std_df = pd.read_csv(f"{result[0].parent / '_'.join(result[0].stem.split('_')[:2])}_noaug_std.csv", index_col=0)
    except FileNotFoundError:
        # Create a DataFrame with zeros if std file not found
        unaugmented_std_df = pd.DataFrame(0, index=unaugmented_df.index, columns=unaugmented_df.columns)

    # Load augmented mean data
    augmented_dfs = []
    augmented_std_dfs = []
    
    # Pair each mean file with its std file
    for r in result:
        mean_df = pd.read_csv(r, index_col=0)
        augmented_dfs.append(mean_df)
        
        # Try to load std file
        try:
            std_df = pd.read_csv(r.parent / f"{r.stem}_std.csv", index_col=0)
            augmented_std_dfs.append(std_df)
        except FileNotFoundError:
            # Create a DataFrame with zeros if std file not found
            augmented_std_dfs.append(pd.DataFrame(0, index=mean_df.index, columns=mean_df.columns))

    # Create a combined DataFrame for all augmenters for each percentage
    aug_all = pd.concat(augmented_dfs)
    
    # Find best augmenter for each data fraction
    best_augs = {}
    best_augs_std = {}
    
    for data_frac in unaugmented_df.index:
        # Filter all augmenter results for this data fraction
        aug_frac_data = []
        for i, df in enumerate(augmented_dfs):
            if data_frac in df.index:
                aug_frac_data.append((i, df.loc[data_frac, METRIC]))
        
        # Find best augmenter (min or max)
        if len(aug_frac_data) > 0:
            if SORT == "MIN":
                best_idx, best_val = min(aug_frac_data, key=lambda x: x[1])
            else:
                best_idx, best_val = max(aug_frac_data, key=lambda x: x[1])
            
            # Store best value and corresponding std
            best_augs[data_frac] = best_val
            if data_frac in augmented_std_dfs[best_idx].index:
                best_augs_std[data_frac] = augmented_std_dfs[best_idx].loc[data_frac, METRIC]
            else:
                best_augs_std[data_frac] = 0  # Default if std not available
        else:
            best_augs[data_frac] = np.nan
            best_augs_std[data_frac] = 0

    # Create DataFrames for plotting
    augmented_best = pd.DataFrame({METRIC: best_augs}, index=unaugmented_df.index)
    std_best = pd.DataFrame({METRIC: best_augs_std}, index=unaugmented_df.index)

    plot = Plot(ax=axes[idx])

    x = np.arange(len(unaugmented_df.index))  # the label locations
    y_aug = augmented_best[METRIC]
    y_noaug = unaugmented_df[METRIC]
    
    # Get standard deviations
    y_aug_std = std_best[METRIC]
    y_noaug_std = unaugmented_std_df[METRIC]

    # Plot with error bars
    plot.ax.bar(x - 0.2, y_noaug, 0.4, label="Baseline", yerr=y_noaug_std, capsize=5)
    plot.ax.bar(x + 0.2, y_aug, 0.4, label="Best Aug.", yerr=y_aug_std, capsize=5)
    plot.ax.legend(loc='upper right')

    plot.ax.set_title(model)
    plot.ax.set_xticks(x)
    plot.ax.set_xticklabels(unaugmented_df.index)
    plot.ax.set_ylim(1, 4) 
    plot.ax.set_ylabel(METRIC)
    plot.ax.set_xlabel("Fraction of train data (75%)")

plt.tight_layout()
plt.show()

# Hat Plot

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from aixchem.plots.core import Plot


def performance_plot(results_path, metric, sort_max=True, drop_index=[], lower_is_better=None):
    # Automatically determine if lower is better based on metric name if not specified
    if lower_is_better is None:
        # Common metrics where lower values are better
        lower_better_metrics = ["RMSE", "MAE", "MSE", "loss", "error"]
        lower_is_better = any(m in metric for m in lower_better_metrics)
    
    # Read all csv files from the dir
    files = [f for f in results_path.iterdir() if f.suffix == ".csv" and not "noaug" in f.stem and not "std" in f.stem and not "FOLD" in f.stem]

    # Group results according to model
    results = {model: [f for f in files if model in f.stem] for model in set([f.stem.split("_")[1] for f in files])}

    # Handle both single and multiple subplot cases
    n_models = len(results.keys())
    fig, axes = plt.subplots(1, n_models, figsize=(4.5*n_models, 4), squeeze=False)
    axes = axes.flatten()  # Flatten in case of single subplot

    for idx, (model, result) in enumerate(results.items()):

        # Get unaugmented results
        unaugmented_df = pd.read_csv(f"{result[0].parent / '_'.join(result[0].stem.split('_')[:2])}_noaug.csv", index_col=0)

        # Get augmented results
        augmented_dfs = [pd.read_csv(r, index_col=0) for r in result]

        # Aggregate to get the best augmented result - invert sort_max if lower is better
        effective_sort = sort_max if not lower_is_better else not sort_max
        
        if effective_sort:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs))).groupby(level=1).max()
        else:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs))).groupby(level=1).min()

        # Ensure both have the same order
        augmented_best = augmented_best.reindex(unaugmented_df.index)

        if drop_index:
            unaugmented_df = unaugmented_df.drop(index=drop_index)
            augmented_best = augmented_best.drop(index=drop_index)

        # Create model plot
        plot = Plot(ax=axes[idx])

        xs = np.arange(len(unaugmented_df.index))
        ys_aug = augmented_best[metric]
        ys_noaug = unaugmented_df[metric]

        # Find the optimal baseline for comparison
        if lower_is_better:
            optimal_baseline = ys_noaug.min()
        else:
            optimal_baseline = ys_noaug.max()

        for x, y_noaug, y_aug in zip(xs, ys_noaug, ys_aug):
            
            # Plot horizontal line to represent base performance
            hline_offset = 0.25
            plot.ax.hlines(y_noaug, x - hline_offset*2, x + hline_offset, color="#A2B1C6", linestyle="-")
            
            # For RMSE: Lower is better, so improvement is when y_aug < y_noaug
            # For R2: Higher is better, so improvement is when y_aug > y_noaug
            is_improvement = (y_aug < y_noaug) if lower_is_better else (y_aug > y_noaug)
            
            # Plot performance difference as bar starting from hline
            bar_color = "#004080" if is_improvement else "#800040"
            plot.ax.bar(x, y_aug - y_noaug, 2*hline_offset, bottom=y_noaug, color=bar_color)

            # Calculate percentage improvement - different formula for different metrics
            if lower_is_better:
                # For RMSE: (noaug - aug)/noaug = percentage reduction
                pct_change = ((y_noaug - y_aug) / y_noaug) * 100 if y_noaug != 0 else 0
            else:
                # For R2: (aug - noaug)/noaug = percentage improvement
                pct_change = ((y_aug - y_noaug) / abs(y_noaug)) * 100 if y_noaug != 0 else 0
            
            # Annotate with percentage improvement
            is_exceeding_optimal = (y_aug < optimal_baseline) if lower_is_better else (y_aug > optimal_baseline)
            font_weight = "bold" if is_exceeding_optimal else "normal"
            
            if is_improvement:
                plot.ax.annotate(f"{pct_change:.1f}%", (x, y_aug), textcoords="offset points", 
                               xytext=(0, -1 if lower_is_better else 7), 
                               ha='center', va='top' if lower_is_better else 'baseline', 
                               fontweight=font_weight, fontsize=10, color=bar_color, rotation=90)
            else:
                plot.ax.annotate(f"{pct_change:.1f}%", (x, y_noaug), textcoords="offset points", 
                               xytext=(0, 7 if lower_is_better else -1), 
                               ha='center', va='baseline' if lower_is_better else 'top', 
                               fontweight=font_weight, fontsize=10, color=bar_color, rotation=90)

        # Add line indicating the optimal performance of the unaugmented model
        plot.ax.hlines(optimal_baseline, xs[0] - 0.5, xs[-1] + 0.5, color="#A2B1C6", linestyle=":")
        
        # Add label for the optimal value
        optimal_label = "Min" if lower_is_better else "Max"
        plot.ax.annotate(f"{optimal_label}: {optimal_baseline:.2f}", (xs[0], optimal_baseline), 
                       textcoords="offset points", xytext=(0, 2), 
                       ha='left', va='bottom', fontsize=10, color="#A2B1C6")

        # layout
        plot.ax.set_title(model)
        plot.ax.set_xticks(xs)
        plot.ax.set_xticklabels(ys_noaug.index)
        
        # Set appropriate y-limits based on data
        y_min = min(min(ys_aug), min(ys_noaug)) * 0.7  # Add some padding
        y_max = max(max(ys_aug), max(ys_noaug)) * 1.2
        plot.ax.set_ylim(y_min, y_max)
        
        plot.ax.set_ylabel(metric)
        plot.ax.set_xlabel("Fraction of train data (75%)")

# Define the path to the results
AIxChem = Path.cwd().parent
RESULTS = AIxChem / "docs/buttar_norrby_results/holdout_RMSE"

performance_plot(
    results_path=RESULTS, 
    metric="holdout_RMSE", 
    sort_max=True,
    lower_is_better=True  # Explicitly set that for RMSE lower is better
)

## t-test

Calculate the statistical difference between the augmented and non-augmented results. If the p-values for a certain fraction are below 0.05 this means that the performances coming from the augmented dataset are statistically better or worse than the non-augmented performance.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel


def get_best_augmented_performance(results_path, metric, sort_max=True, drop_index=[]):
    """
    This function goes through the folder where the mean results are stored and gets the best augmenter per fraction of dataset considered for all the different models

    :param results_path: path to the folder where the mean results (CSV files) are stored
    :param metric: metric used to sort the performances of the models

    Returns
    - best_performances: it is a dictionary that has as top-keys the ML models. Second level keys are the percentage of the data used, to each of these keys corresponds 
                         two values: name of the best augmenter, mean_value
    """
    # Read all csv files from the dir
    files = [f for f in results_path.iterdir() if f.suffix == ".csv" and not "noaug" in f.stem and not "std" in f.stem and not "FOLD" in f.stem]

    # Group results according to model
    results = {model: [f for f in files if model in f.stem] for model in set([f.stem.split("_")[1] for f in files])}

    best_performances = {}

    for model, result in results.items():
        # Get unaugmented results
        unaugmented_df = pd.read_csv(f"{result[0].parent / '_'.join(result[0].stem.split('_')[:2])}_noaug.csv", index_col=0)

        # Get augmented results
        augmented_dfs = [pd.read_csv(r, index_col=0) for r in result]

        # Aggregate to get the best augmented result
        if sort_max:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs)), names=['file_index', 'row_index']).groupby(level=1).max()
        else:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs)), names=['file_index', 'row_index']).groupby(level=1).min()

        # Ensure both have the same order
        augmented_best = augmented_best.reindex(unaugmented_df.index)

        # drop the specified indexes
        if drop_index:
            print("Indexes to drop:", drop_index)
            unaugmented_df = unaugmented_df.drop(index=drop_index)
            augmented_best = augmented_best.drop(index=drop_index)

        best_performances[model] = {}
        # iterate through the percentage of augmented best
        for idx in augmented_best.index:
            # Get the best value for the current percentage
            best_value = augmented_best.loc[idx, metric]
            best_file_index = None
            best_file_name = None

            # Iterate through the augmented DataFrames to find the file that contains the best value
            for file_index, df in enumerate(augmented_dfs):
                
                # Check if the current percentage is in the DataFrame and if the value matches the best value
                if idx in df.index and df.loc[idx, metric] == best_value:
                    best_file_index = file_index
                    # Keep only the first part of the file name
                    best_file_name = result[file_index].stem.split("_aug")[0] 
                    break
             # If the best file index was found, store the best value and file name in the dictionary
            if best_file_index is not None:
                best_performances[model][idx] = (best_value, best_file_name)
            else:
                print(f"Warning: Best file for index {idx} not found in model {model}.")

    return best_performances


def get_performance_per_fold(folds_path, metric, best_performances, n_folds: int):
    """
    This function, according to the information stored in "best_performances", creates a dictionary where the results per model/percentage/best mean sequences are stored.
    """
    noaug_per_fold = {}
    aug_per_fold = {}

    # iterate through the top level keys (models).
    for model in best_performances.keys():
        noaug_per_fold[model] = {}
        aug_per_fold[model] = {}
        
        # Get the folder list
        fold_list = [f"FOLD{i}" for i in range(n_folds)]
        
        # Get file lists once outside the loops
        noaug_files = [f for f in folds_path.iterdir() if f.suffix == ".csv" and "noaug" in f.stem and "FOLD" in f.stem]
        
        # Read the non-augmented data file once
        if noaug_files:
            noaug_df = pd.read_csv(noaug_files[0], index_col=0)
            
        # Process each percentage
        for percentage in best_performances[model]:
            if percentage not in noaug_per_fold[model]:
                noaug_per_fold[model][percentage] = {}
            if percentage not in aug_per_fold[model]:
                aug_per_fold[model][percentage] = {}
                
            # Get the augmenter dataset info
            performance, dataset = best_performances[model][percentage]
            
            # Get the augmented data file for this dataset
            aug_files = [f for f in folds_path.iterdir() if f.suffix == ".csv" and dataset in f.stem and not "noaug" in f.stem and "FOLD" in f.stem]
            
            # Read the augmented data file once
            if aug_files:
                aug_df = pd.read_csv(aug_files[0], index_col=0)
            
            # For each fold, extract the fold-specific data
            for fold in fold_list:

                if noaug_files:

                    mask_noaug = (noaug_df['FOLD'] == fold) & (noaug_df.index.str.contains(percentage))
                    filtered_noaug = noaug_df[mask_noaug]
                    
                    if not filtered_noaug.empty:
                        performance_noaug = filtered_noaug[metric].iloc[0]  # Take first match
                        noaug_per_fold[model][percentage][fold] = performance_noaug
                

                if aug_files:
                    mask_aug = (aug_df['FOLD'] == fold) & (aug_df.index.str.contains(percentage))
                    filtered_aug = aug_df[mask_aug]
                    
                    if not filtered_aug.empty:
                        performance_aug = filtered_aug[metric].iloc[0]  # Take first match
                        aug_per_fold[model][percentage][fold] = performance_aug


    return noaug_per_fold, aug_per_fold
                            

def calculate_p_values(noaug_per_fold, aug_per_fold, lower_is_better=True):
    """
    Calculate one-tailed p-values to compare the performance of augmented and non-augmented data.
    
    For metrics where lower is better (e.g., RMSE, MSE):
    - We want to test if augmented is better than non-augmented (aug < noaug)
    
    For metrics where higher is better (e.g., R², accuracy):
    - We want to test if augmented is better than non-augmented (aug > noaug)
    
    :param noaug_per_fold: Dictionary containing results for non-augmented data
    :param aug_per_fold: Dictionary containing results for augmented data
    :param lower_is_better: Whether lower values indicate better performance (default True for RMSE-like metrics)
    
    Returns:
    - p_values: Dictionary with models as keys and another dictionary as values, 
                where the inner dictionary has percentages as keys and p-values as values.
    - t_stats: Dictionary with the t-statistics for each comparison.
    """
    
    p_values = {}
    t_stats = {}  # To store t-statistics
    
    # Iterate over each model in the noaug_per_fold dictionary
    for model in noaug_per_fold.keys():
        p_values[model] = {}
        t_stats[model] = {}
        
        # Iterate over each percentage in the noaug_per_fold dictionary for the current model
        for percentage in noaug_per_fold[model].keys():
            # Extract the noaug values for the current model and percentage across all folds
            noaug_values = [noaug_per_fold[model][percentage][fold] for fold in noaug_per_fold[model][percentage].keys()]
            # Extract the augmented values for the current model and percentage across all folds
            aug_values = [aug_per_fold[model][percentage][fold] for fold in aug_per_fold[model][percentage].keys()]

            # Ensure both noaug_values and aug_values have more than one element to perform the test
            if len(noaug_values) > 1 and len(aug_values) > 1:
                # Calculate the t-statistic and two-tailed p-value using a paired t-test
                # For RMSE (lower_is_better=True): noaug - aug > 0 means aug is better (lower)
                # For R² (lower_is_better=False): noaug - aug < 0 means aug is better (higher)
                t_stat, two_tailed_p_value = ttest_rel(noaug_values, aug_values)
                
                # Determine if we're testing if augmentation is better
                # For lower_is_better=True: we want to test if aug < noaug (positive t-stat)
                # For lower_is_better=False: we want to test if aug > noaug (negative t-stat)
                if lower_is_better:
                    # One-tailed p-value for H₁: μ₁ > μ₂ (noaug > aug)
                    one_tailed_p_value = two_tailed_p_value / 2 if t_stat > 0 else 1 - (two_tailed_p_value / 2)
                else:
                    # One-tailed p-value for H₁: μ₁ < μ₂ (noaug < aug)
                    one_tailed_p_value = two_tailed_p_value / 2 if t_stat < 0 else 1 - (two_tailed_p_value / 2)
                
                # Store the one-tailed p-value and t-stat in the respective dictionaries
                p_values[model][percentage] = one_tailed_p_value
                t_stats[model][percentage] = t_stat
            else:
                # Handle case where there are not enough data points to run the test
                p_values[model][percentage] = None
                t_stats[model][percentage] = None
    
    return p_values, t_stats


def plot_p_values(p_values):
    """
    Plot p-values for each model in a single figure with multiple subplots.

    - p_values: Dictionary with models as keys and another dictionary as values, 
                where the inner dictionary has percentages as keys and p-values as values.
    """
    # Sort the models alphabetically
    sorted_models = sorted(p_values.keys())
    
    # Determine the number of models
    n_models = len(sorted_models)  # Changed from num_models to n_models for consistency
    
    # Create figure with appropriate number of subplots
    fig, axes = plt.subplots(1, n_models, figsize=(4.5*n_models, 4), squeeze=False)
    axes = axes.flatten()  # Flatten in case of single subplot
    
    # Plot each model's p-values in a separate subplot
    for idx, model in enumerate(sorted_models):
        percentages = p_values[model]
        
        # Filter out None values
        valid_percentages = {k: v for k, v in percentages.items() if v is not None}
        
        ax = axes[idx]
        ax.bar(valid_percentages.keys(), valid_percentages.values())
        ax.axhline(y=0.05, color='r', linestyle='--', label='Significance Threshold (0.05)')
        ax.set_xlabel('Percentage')
        ax.set_ylabel('P-value')
        ax.set_title(model)
        ax.legend()
    
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()


def p_values_to_csv(p_values, output_path):
    """
    Save p-values to a CSV file.

    - p_values: Dictionary with models as keys and another dictionary as values, 
                where the inner dictionary has percentages as keys and p-values as values.
    - output_path: Path to save the CSV file.
    """
    # Create a DataFrame from the p_values dictionary
    p_values_df = pd.DataFrame.from_dict({(model, percentage): p_value for model, percentages in p_values.items() for percentage, p_value in percentages.items()}, orient='index', columns=['P-value'])
    
    # Reset index to have model and percentage as columns
    p_values_df.reset_index(inplace=True)
    
    # Rename columns
    p_values_df.columns = ['Model_Percentage', 'P-value']
    
    # Split the 'Model_Percentage' column into 'Model' and 'Percentage'
    p_values_df[['Model', 'Percentage']] = pd.DataFrame(p_values_df['Model_Percentage'].tolist(), index=p_values_df.index)
    
    # Drop the 'Model_Percentage' column
    p_values_df.drop(columns=['Model_Percentage'], inplace=True)
    
    # Reorder columns
    p_values_df = p_values_df[['Model', 'Percentage', 'P-value']]
    
    # Save to CSV
    p_values_df.to_csv(output_path, index=False)



# Define the path to the results
AIxChem = Path.cwd().parent
results = AIxChem / "docs/buttar_norrby_results/holdout_RMSE"
# scoring metric
metric = "holdout_RMSE"
# number of folds
n_folds = 20

best_performances = get_best_augmented_performance(results_path=results, metric=metric)

noaug_per_fold, aug_per_fold = get_performance_per_fold(folds_path=results, metric=metric, best_performances=best_performances, n_folds=n_folds)

p_values, t_stat = calculate_p_values(noaug_per_fold=noaug_per_fold, aug_per_fold=aug_per_fold, lower_is_better=True)
                                                 
plot_p_values(p_values=p_values)

# 2 -steps TOST
## Step 1. 
We are comparing the not-augmented results with themself, in order to understand when is the Plateau reached. 
In this example, it is possible to observe that RMSE values coming from 90% oh training size is statistically indistinguishable from the RMSE values coming from 100% of the training size.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import ttost_ind


def get_best_augmented_performance(results_path, metric, sort_max=True, drop_index=[]):
    """
    This function goes through the folder where the mean results are stored and gets the best augmenter per fraction of dataset considered for all the different models

    :param results_path: path to the folder where the mean results (CSV files) are stored
    :param metric: metric used to sort the performances of the models

    Returns
    - best_performances: it is a dictionary that has as top-keys the ML models. Second level keys are the percentage of the data used, to each of these keys corresponds 
                         two values: name of the best augmenter, mean_value
    """
    # Read all csv files from the dir
    files = [f for f in results_path.iterdir() if f.suffix == ".csv" and not "noaug" in f.stem and not "std" in f.stem and not "FOLD" in f.stem]

    # Group results according to model
    results = {model: [f for f in files if model in f.stem] for model in set([f.stem.split("_")[1] for f in files])}

    best_performances = {}

    for model, result in results.items():
        # Get unaugmented results
        unaugmented_df = pd.read_csv(f"{result[0].parent / '_'.join(result[0].stem.split('_')[:2])}_noaug.csv", index_col=0)

        # Get augmented results
        augmented_dfs = [pd.read_csv(r, index_col=0) for r in result]

        # Aggregate to get the best augmented result
        if sort_max:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs)), names=['file_index', 'row_index']).groupby(level=1).max()
        else:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs)), names=['file_index', 'row_index']).groupby(level=1).min()

        # Ensure both have the same order
        augmented_best = augmented_best.reindex(unaugmented_df.index)

        # drop the specified indexes
        if drop_index:
            print("Indexes to drop:", drop_index)
            unaugmented_df = unaugmented_df.drop(index=drop_index)
            augmented_best = augmented_best.drop(index=drop_index)

        best_performances[model] = {}
        # iterate through the percentage of augmented best
        for idx in augmented_best.index:
            # Get the best value for the current percentage
            best_value = augmented_best.loc[idx, metric]
            best_file_index = None
            best_file_name = None

            # Iterate through the augmented DataFrames to find the file that contains the best value
            for file_index, df in enumerate(augmented_dfs):
                
                # Check if the current percentage is in the DataFrame and if the value matches the best value
                if idx in df.index and df.loc[idx, metric] == best_value:
                    best_file_index = file_index
                    # Keep only the first part of the file name
                    best_file_name = result[file_index].stem.split("_aug")[0] 
                    break
             # If the best file index was found, store the best value and file name in the dictionary
            if best_file_index is not None:
                best_performances[model][idx] = (best_value, best_file_name)
            else:
                print(f"Warning: Best file for index {idx} not found in model {model}.")

    return best_performances

def best_performance_per_model(best_unaugmented_performances, metric):
    """
    Identifies the best performance value for each model across all data fractions.
    
    Parameters:
    - best_unaugmented_performances: Dictionary with models as keys and another dictionary 
      as values, where inner dictionaries have percentages/fractions as keys and 
      performance values as values.
    - metric: Performance metric to optimize (e.g., "holdout_R2", "holdout_RMSE", "holdout_MAE")
    
    Returns:
    - best_performance_unaug: Dictionary with models as keys and their best performance 
      value as values, optimized according to the metric direction.
    """
    best_performance_unaug = {}
    
    if metric == "holdout_R2":
        # For R², higher values are better
        for model in best_unaugmented_performances.keys():
            best_performance_unaug[model] = max(best_unaugmented_performances[model].values())

    elif metric == "holdout_RMSE" or metric == "holdout_MAE":
        # For RMSE and MAE, lower values are better
        for model in best_unaugmented_performances.keys():
            best_performance_unaug[model] = min(best_unaugmented_performances[model].values())

    return best_performance_unaug

def get_performance_per_fold(folds_path, metric, best_performances, n_folds: int):
    """
    This function, according to the information stored in "best_performances", creates a dictionary where the results per model/percentage/best mean sequences are stored.
    """
    noaug_per_fold = {}
    aug_per_fold = {}

    # iterate through the top level keys (models).
    for model in best_performances.keys():
        noaug_per_fold[model] = {}
        aug_per_fold[model] = {}
        
        # Get the folder list
        fold_list = [f"FOLD{i}" for i in range(n_folds)]
        
        # Get file lists once outside the loops
        noaug_files = [f for f in folds_path.iterdir() if f.suffix == ".csv" and "noaug" in f.stem and "FOLD" in f.stem]
        
        # Read the non-augmented data file once
        if noaug_files:
            noaug_df = pd.read_csv(noaug_files[0], index_col=0)
            
        # Process each percentage
        for percentage in best_performances[model]:
            if percentage not in noaug_per_fold[model]:
                noaug_per_fold[model][percentage] = {}
            if percentage not in aug_per_fold[model]:
                aug_per_fold[model][percentage] = {}
                
            # Get the augmenter dataset info
            performance, dataset = best_performances[model][percentage]
            
            # Get the augmented data file for this dataset
            aug_files = [f for f in folds_path.iterdir() if f.suffix == ".csv" and dataset in f.stem and not "noaug" in f.stem and "FOLD" in f.stem]
            
            # Read the augmented data file once
            if aug_files:
                aug_df = pd.read_csv(aug_files[0], index_col=0)
            
            # For each fold, extract the fold-specific data
            for fold in fold_list:

                if noaug_files:

                    mask_noaug = (noaug_df['FOLD'] == fold) & (noaug_df.index.str.contains(percentage))
                    filtered_noaug = noaug_df[mask_noaug]
                    
                    if not filtered_noaug.empty:
                        performance_noaug = filtered_noaug[metric].iloc[0]  # Take first match
                        noaug_per_fold[model][percentage][fold] = performance_noaug
                

                if aug_files:
                    mask_aug = (aug_df['FOLD'] == fold) & (aug_df.index.str.contains(percentage))
                    filtered_aug = aug_df[mask_aug]
                    
                    if not filtered_aug.empty:
                        performance_aug = filtered_aug[metric].iloc[0]  # Take first match
                        aug_per_fold[model][percentage][fold] = performance_aug


    return noaug_per_fold, aug_per_fold

def get_noaug_per_fold100(folds_path, metric, best_performances, n_folds: int):
    """
    This function creates a dictionary with unaugmented performance at 100% data fraction
    for each model and percentage combination across all folds.
    
    :param folds_path: path to the folder where the results CSV is stored
    :param metric: metric used to evaluate model performance
    :param best_performances: dictionary coming from get_best_augmented_performance
    :param n_folds: number of folds in the dataset
    
    Returns:
    - noaug_per_fold100: dictionary with models as top-keys, percentages as second-level keys,
                         and fold-specific performance values at 100% data fraction
    """
    noaug_per_fold100 = {}
    
    # Find the single noaug file with FOLD data
    noaug_files = [f for f in folds_path.iterdir() if f.suffix == ".csv" and "noaug" in f.stem and "FOLD" in f.stem]
    
    if not noaug_files:
        print("No noaug FOLD files found.")
        return noaug_per_fold100
    
    # Read the single CSV containing all fold data
    noaug_df = pd.read_csv(noaug_files[0], index_col=0)
    
    # Get list of fold identifiers
    fold_list = [f"FOLD{i}" for i in range(n_folds)]
    
    # Iterate through the models
    for model in best_performances.keys():
        noaug_per_fold100[model] = {}
        
        # Iterate through the percentages for the current model
        for percentage in best_performances[model]:
            noaug_per_fold100[model][percentage] = {}
            
            # Filter the dataframe for 100% data fraction rows that match the model
            mask_100 = (noaug_df.index.str.contains("100%")) & (noaug_df.index.str.contains(model))
            filtered_100 = noaug_df[mask_100]
            
            if filtered_100.empty:
                print(f"No 100% data fraction found for model {model}")
                continue
            
            # For each fold, extract the performance value at 100% data fraction
            for fold in fold_list:
                fold_mask = filtered_100['FOLD'] == fold
                fold_data = filtered_100[fold_mask]
                
                if not fold_data.empty:
                    performance_noaug = fold_data[metric].iloc[0]  # Take first match
                    noaug_per_fold100[model][percentage][fold] = performance_noaug
    
    return noaug_per_fold100

def get_std_unaugmented_performances(mean_path, metric):
    """
    Get the standard deviation of the unaugmented performances for each model
    
    :param mean_path: path to the folder where the results are stored (CSV files)
    :param metric: metric used to evaluate model performance
    """
    std_noaug = {}
    
    # Get only files with pattern AGN_{model}_noaug_std.csv
    noaug_files = [f for f in mean_path.iterdir() 
                   if f.suffix == ".csv" and 
                   f.stem.startswith("AGN_") and 
                   "noaug_std" in f.stem]
    
    # Process each file to extract the model name
    for file_path in noaug_files:
        # Extract model name - it's the second part after splitting by "_"
        parts = file_path.stem.split("_")
        if len(parts) >= 2:
            model = parts[1]  # Get model name (RF, SVM, etc.)
            
            # Read the CSV file
            noaug_df = pd.read_csv(file_path, index_col=0)
            
            if "100%" in noaug_df.index:
                # Get the standard deviation for the unaugmented model at 100% fraction
                performance_noaug = noaug_df.loc["100%", metric]
                
                # Store the performance in the dictionary
                std_noaug[model] = performance_noaug
    
    return std_noaug
                            
def perform_tost(noaug_per_fold100, noaug_per_fold, metric, standard_deviation):
    """
    Perform the Two One-Sided T-Test (TOST), also known as the equivalence test.
    Skip comparisons when percentages are 100% (to avoid comparing identical distributions).
    
    Null hypothesis: m1 - m2 < low or m1 - m2 > upp 
    Alternative hypothesis: low < m1 - m2 < upp
    
    :param noaug_per_fold100: dictionary with the unaugmented performances at 100%
    :param noaug_per_fold: dictionary with the unaugmented performances at various percentages
    :param metric: the metric to be used for the TOST
    :param standard_deviation: dictionary with standard deviation for each model
    """
    p_values = {}

    # iterate through the models
    for model in noaug_per_fold100.keys():
        p_values[model] = {}

        # Calculate the lower and upper bounds for the TOST
        lower_bound = -(standard_deviation[model])
        upper_bound = standard_deviation[model]

        # iterate through the percentages
        for percentage in noaug_per_fold100[model].keys():
            # Skip the 100% comparison with itself
            if percentage == "100%":
                p_values[model][percentage] = None  # or np.nan or any other marker
                continue
                
            # Extract the noaug values for the current model and percentage across all folds
            noaug_values = [noaug_per_fold100[model][percentage][fold] for fold in noaug_per_fold100[model][percentage].keys()]
            # Extract the augmented values for the current model and percentage across all folds
            aug_values = [noaug_per_fold[model][percentage][fold] for fold in noaug_per_fold[model][percentage].keys()]

            # perform the TOST
            p_value, pv1, pv2 = ttost_ind(noaug_values, aug_values, low=lower_bound, upp=upper_bound, usevar='unequal')

            # store the p-value
            p_values[model][percentage] = p_value

    return p_values, pv1, pv2

def plot_p_values(p_values):
    """
    Plot p-values for each model in a single figure with multiple subplots.
    Handles both single and multiple models.

    - p_values: Dictionary with models as keys and another dictionary as values, 
                where the inner dictionary has percentages as keys and p-values as values.
    """
    # Sort the models alphabetically
    sorted_models = sorted(p_values.keys())
    
    # Determine the number of models
    num_models = len(sorted_models)
    
    # Create a figure with subplots in one row
    fig, axes = plt.subplots(1, num_models, figsize=(5 * num_models, 5), sharey=True)
    
    # Handle the case of a single model (axes is not an array in this case)
    if num_models == 1:
        axes = [axes]  # Convert to a list with a single element for consistent indexing
    
    # Plot each model's p-values in a separate subplot
    for idx, model in enumerate(sorted_models):
        percentages = p_values[model]
        
        # Filter out None values
        valid_percentages = {k: v for k, v in percentages.items() if v is not None}
        
        ax = axes[idx]
        ax.bar(valid_percentages.keys(), valid_percentages.values())
        ax.axhline(y=0.05, color='r', linestyle='--', label='Significance Threshold (0.05)')
        ax.set_xlabel('Percentage')
        ax.set_ylabel('P-value')
        ax.set_title(model)
        ax.legend()
    
    # Adjust layout
    style = Path.cwd().parent / "aixchem/plots/style.mpl"
    plt.style.use(style)
    plt.rcParams['font.monospace'] = 'DejaVu Sans Mono'
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()

def p_values_to_csv(p_values, output_path):
    """
    Save p-values to a CSV file.

    - p_values: Dictionary with models as keys and another dictionary as values, 
                where the inner dictionary has percentages as keys and p-values as values.
    - output_path: Path to save the CSV file.
    """
    # Create a DataFrame from the p_values dictionary
    p_values_df = pd.DataFrame.from_dict({(model, percentage): p_value for model, percentages in p_values.items() for percentage, p_value in percentages.items()}, orient='index', columns=['P-value'])
    
    # Reset index to have model and percentage as columns
    p_values_df.reset_index(inplace=True)
    
    # Rename columns
    p_values_df.columns = ['Model_Percentage', 'P-value']
    
    # Split the 'Model_Percentage' column into 'Model' and 'Percentage'
    p_values_df[['Model', 'Percentage']] = pd.DataFrame(p_values_df['Model_Percentage'].tolist(), index=p_values_df.index)
    
    # Drop the 'Model_Percentage' column
    p_values_df.drop(columns=['Model_Percentage'], inplace=True)
    
    # Reorder columns
    p_values_df = p_values_df[['Model', 'Percentage', 'P-value']]
    
    # Save to CSV
    p_values_df.to_csv(output_path, index=False)

# Define the path to the results
AIxChem = Path.cwd().parent
results = AIxChem / "docs/buttar_norrby_results/holdout_RMSE"
# # scoring metric
metric = "holdout_RMSE"
# number of folds
n_folds = 20

best_performances = get_best_augmented_performance(results_path=results, metric=metric)

std_noaug = get_std_unaugmented_performances(mean_path=results, metric=metric)

noaug_per_fold, aug_per_fold = get_performance_per_fold(folds_path=results, metric=metric, best_performances=best_performances, n_folds=n_folds)

noaug_per_fold100 = get_noaug_per_fold100(folds_path=results, metric=metric, best_performances=best_performances, n_folds=n_folds)

p_values, pv1, pv2 = perform_tost(noaug_per_fold100=noaug_per_fold100, noaug_per_fold=noaug_per_fold, metric=metric, standard_deviation=std_noaug)

print(p_values)
                                                    
plot_p_values(p_values=p_values)

# 2-Steps TOST
## Second Step

In this example we are comparing the RMSE values coming from 90% of the non-augmented training size with the RMSE values coming from the different fractions of the augmented dataset.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.weightstats import ttost_ind


def get_best_augmented_performance(results_path, metric, sort_max=True, drop_index=[]):
    """
    This function goes through the folder where the mean results are stored and gets the best augmenter per fraction of dataset considered for all the different models

    :param results_path: path to the folder where the mean results (CSV files) are stored
    :param metric: metric used to sort the performances of the models

    Returns
    - best_performances: it is a dictionary that has as top-keys the ML models. Second level keys are the percentage of the data used, to each of these keys corresponds 
                         two values: name of the best augmenter, mean_value
    """
    # Read all csv files from the dir
    files = [f for f in results_path.iterdir() if f.suffix == ".csv" and not "noaug" in f.stem and not "std" in f.stem and not "FOLD" in f.stem]

    # Group results according to model
    results = {model: [f for f in files if model in f.stem] for model in set([f.stem.split("_")[1] for f in files])}

    best_performances = {}

    for model, result in results.items():
        # Get unaugmented results
        unaugmented_df = pd.read_csv(f"{result[0].parent / '_'.join(result[0].stem.split('_')[:2])}_noaug.csv", index_col=0)

        # Get augmented results
        augmented_dfs = [pd.read_csv(r, index_col=0) for r in result]

        # Aggregate to get the best augmented result
        if sort_max:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs)), names=['file_index', 'row_index']).groupby(level=1).max()
        else:
            augmented_best = pd.concat(augmented_dfs, keys=range(len(augmented_dfs)), names=['file_index', 'row_index']).groupby(level=1).min()

        # Ensure both have the same order
        augmented_best = augmented_best.reindex(unaugmented_df.index)

        # drop the specified indexes
        if drop_index:
            print("Indexes to drop:", drop_index)
            unaugmented_df = unaugmented_df.drop(index=drop_index)
            augmented_best = augmented_best.drop(index=drop_index)

        best_performances[model] = {}
        # iterate through the percentage of augmented best
        for idx in augmented_best.index:
            # Get the best value for the current percentage
            best_value = augmented_best.loc[idx, metric]
            best_file_index = None
            best_file_name = None

            # Iterate through the augmented DataFrames to find the file that contains the best value
            for file_index, df in enumerate(augmented_dfs):
                
                # Check if the current percentage is in the DataFrame and if the value matches the best value
                if idx in df.index and df.loc[idx, metric] == best_value:
                    best_file_index = file_index
                    # Keep only the first part of the file name
                    best_file_name = result[file_index].stem.split("_aug")[0] 
                    break
             # If the best file index was found, store the best value and file name in the dictionary
            if best_file_index is not None:
                best_performances[model][idx] = (best_value, best_file_name)
            else:
                print(f"Warning: Best file for index {idx} not found in model {model}.")

    return best_performances

def get_performance_per_fold(folds_path, metric, best_performances, n_folds: int, best_percentage_per_model: dict):
    """
    This function extracts fold-specific performance values from a single CSV file 
    where different folds are identified by a 'FOLD' column.
    
    :param folds_path: path to the folder containing CSV files with 'FOLD' column
    :param metric: metric used to evaluate model performance
    :param best_performances: dictionary from get_best_augmented_performance
    :param n_folds: number of folds in the dataset
    :param best_percentage_per_model: dictionary with best percentage fraction for each model
    
    Returns:
    - noaug_per_fold: dictionary with unaugmented performance values per model/percentage/fold
    - aug_per_fold: dictionary with augmented performance values per model/percentage/fold
    """
    noaug_per_fold = {}
    aug_per_fold = {}
    
    # Find the single noaug file with FOLD data
    noaug_files = [f for f in folds_path.iterdir() if f.suffix == ".csv" and "noaug" in f.stem and "FOLD" in f.stem]
    
    if not noaug_files:
        print("No noaug FOLD file found.")
        return noaug_per_fold, aug_per_fold
        
    # Read the noaug CSV file
    noaug_df = pd.read_csv(noaug_files[0], index_col=0)
    
    # Generate fold identifiers
    fold_list = [f"FOLD{i}" for i in range(n_folds)]
    
    # iterate through the models
    for model in best_performances.keys():
        noaug_per_fold[model] = {}
        aug_per_fold[model] = {}
        
        # Get the best percentage for this model
        best_frac = best_percentage_per_model.get(model)
        
        # Process each percentage from best_performances
        for percentage in best_performances[model]:
            noaug_per_fold[model][percentage] = {}
            aug_per_fold[model][percentage] = {}
            
            # Get augmenter information
            _, dataset = best_performances[model][percentage]
            
            # Find the augmented data file for this dataset
            aug_files = [f for f in folds_path.iterdir() 
                        if f.suffix == ".csv" and dataset in f.stem and 
                        not "noaug" in f.stem and "FOLD" in f.stem]
                
            if not aug_files:
                print(f"No augmented FOLD file found for dataset {dataset}")
                continue
                
            # Read the augmented data file
            aug_df = pd.read_csv(aug_files[0], index_col=0)
            
            # Process each fold
            for fold in fold_list:
                # For unaugmented data: filter by fold and model, use best_frac percentage
                noaug_mask = (
                    (noaug_df['FOLD'] == fold) & 
                    (noaug_df.index.str.contains(model)) &
                    (noaug_df.index.str.contains(best_frac))
                )
                
                noaug_filtered = noaug_df[noaug_mask]
                
                if not noaug_filtered.empty:
                    performance_noaug = noaug_filtered[metric].iloc[0]
                    noaug_per_fold[model][percentage][fold] = performance_noaug
                else:
                    print(f"No unaugmented data found for model {model}, fold {fold}, fraction {best_frac}")
                
                # For augmented data: filter by fold, model, and percentage
                aug_mask = (
                    (aug_df['FOLD'] == fold) & 
                    (aug_df.index.str.contains(model)) &
                    (aug_df.index.str.contains(percentage))
                )
                
                aug_filtered = aug_df[aug_mask]
                
                if not aug_filtered.empty:
                    performance_aug = aug_filtered[metric].iloc[0]
                    aug_per_fold[model][percentage][fold] = performance_aug
                else:
                    print(f"No augmented data found for model {model}, fold {fold}, percentage {percentage}")
    
    return noaug_per_fold, aug_per_fold

def get_std_unaugmented_performances(mean_path, metric, best_percentage_per_model: dict):
    """
    Get the standard deviation of unaugmented performances for each model
    
    :param mean_path: path to the folder where the standard deviation CSV files are stored
    :param metric: metric used to evaluate model performance
    :param best_percentage_per_model: dictionary with best percentage fraction for each model
    """
    std_noaug = {}
    
    # Get only files with pattern AGN_{model}_noaug_std.csv
    noaug_files = [f for f in mean_path.iterdir() 
                  if f.suffix == ".csv" and 
                  f.stem.startswith("AGN_") and 
                  "noaug_std" in f.stem]
    
    print("Found std files:", noaug_files)
    
    # Process each file to extract the model name
    for file_path in noaug_files:
        # Extract model name - it's the second part after splitting by "_"
        parts = file_path.stem.split("_")
        if len(parts) >= 2:
            model = parts[1]  # Get model name (RF, SVM, etc.)
            
            # Read the CSV file
            noaug_df = pd.read_csv(file_path, index_col=0)
            
            # Get the best fraction for this model
            fraction = best_percentage_per_model.get(model)
            
            if fraction in noaug_df.index:
                # Get standard deviation for the selected percentage fraction
                performance_noaug = noaug_df.loc[fraction, metric]
                
                # Store the standard deviation in the dictionary
                std_noaug[model] = performance_noaug
            else:
                print(f"Warning: Model {model} doesn't have fraction {fraction} in std file")
    
    return std_noaug
                            
def perform_tost(noaug_per_fold, aug_per_fold, metric, standard_deviation):
    """
    Perform the Two One-Sided T-Test (TOST), also known as the equivalence test. 
    
    Null hypothesis: m1 - m2 < low or m1 - m2 > upp alternative hypothesis: low < m1 - m2 < upp
    
    :param noaug_per_fold: dictionary with the unaugmented performances
    :param aug_per_fold: dictionary with the augmented performances
    :param metric: the metric to be used for the TOST
    :param best_performance: dictionary with the best performance for each model
    """

    p_values = {}

    # iterate through the models
    for model in noaug_per_fold.keys():
        p_values[model] = {}

        # Calculate the lower and upper bounds for the TOST
        lower_bound = -(standard_deviation[model])
        upper_bound = standard_deviation[model]

        # iterate through the percentages
        for percentage in noaug_per_fold[model].keys():
            # Extract the noaug values for the current model and percentage across all folds
            noaug_values = [noaug_per_fold[model][percentage][fold] for fold in noaug_per_fold[model][percentage].keys()]
            # Extract the augmented values for the current model and percentage across all folds
            aug_values = [aug_per_fold[model][percentage][fold] for fold in aug_per_fold[model][percentage].keys()]

            # perform the TOST
            p_value, pv1, pv2 = ttost_ind(noaug_values, aug_values, low=lower_bound, upp=upper_bound, usevar='unequal')

            # store the p-value
            p_values[model][percentage] = p_value

    return p_values, pv1, pv2

def plot_p_values(p_values):
    """
    Plot p-values for each model in a single figure with multiple subplots.
    Handles both single and multiple models.

    - p_values: Dictionary with models as keys and another dictionary as values, 
                where the inner dictionary has percentages as keys and p-values as values.
    """
    # Sort the models alphabetically
    sorted_models = sorted(p_values.keys())
    
    # Determine the number of models
    num_models = len(sorted_models)
    
    # Create a figure with subplots in one row
    fig, axes = plt.subplots(1, num_models, figsize=(5 * num_models, 5), sharey=True)
    
    # Handle the case of a single model (axes is not an array in this case)
    if num_models == 1:
        axes = [axes]  # Convert to a list with a single element for consistent indexing
    
    # Plot each model's p-values in a separate subplot
    for idx, model in enumerate(sorted_models):
        percentages = p_values[model]
        
        # Filter out None values
        valid_percentages = {k: v for k, v in percentages.items() if v is not None}
        
        ax = axes[idx]
        ax.bar(valid_percentages.keys(), valid_percentages.values())
        ax.axhline(y=0.05, color='r', linestyle='--', label='Significance Threshold (0.05)')
        ax.set_xlabel('Percentage')
        ax.set_ylabel('P-value')
        ax.set_title(model)
        ax.legend()
    
    # Adjust layout
    # Define the path to the results
    style = Path.cwd().parent / "aixchem/plots/style.mpl"
    plt.style.use(style)
    plt.rcParams['font.monospace'] = 'DejaVu Sans Mono'
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()

def p_values_to_csv(p_values, output_path):
    """
    Save p-values to a CSV file.

    - p_values: Dictionary with models as keys and another dictionary as values, 
                where the inner dictionary has percentages as keys and p-values as values.
    - output_path: Path to save the CSV file.
    """
    # Create a DataFrame from the p_values dictionary
    p_values_df = pd.DataFrame.from_dict({(model, percentage): p_value for model, percentages in p_values.items() for percentage, p_value in percentages.items()}, orient='index', columns=['P-value'])
    
    # Reset index to have model and percentage as columns
    p_values_df.reset_index(inplace=True)
    
    # Rename columns
    p_values_df.columns = ['Model_Percentage', 'P-value']
    
    # Split the 'Model_Percentage' column into 'Model' and 'Percentage'
    p_values_df[['Model', 'Percentage']] = pd.DataFrame(p_values_df['Model_Percentage'].tolist(), index=p_values_df.index)
    
    # Drop the 'Model_Percentage' column
    p_values_df.drop(columns=['Model_Percentage'], inplace=True)
    
    # Reorder columns
    p_values_df = p_values_df[['Model', 'Percentage', 'P-value']]
    
    # Save to CSV
    p_values_df.to_csv(output_path, index=False)


# insert the path to the results
AIxChem = Path.cwd().parent
results = AIxChem / "docs/buttar_norrby_results/holdout_RMSE"

# scoring metric
metric = "holdout_RMSE"
# fraction Plateau
fraction_plateau = {"RF": "90%"}
# number of folds
n_folds = 20

best_performances = get_best_augmented_performance(results_path=results, metric=metric)

std_noaug = get_std_unaugmented_performances(mean_path=results, metric=metric, best_percentage_per_model=fraction_plateau)

noaug_per_fold, aug_per_fold = get_performance_per_fold(folds_path=results, metric=metric, best_performances=best_performances, n_folds=n_folds, best_percentage_per_model=fraction_plateau)

p_values, pv1, pv2 = perform_tost(noaug_per_fold=noaug_per_fold, aug_per_fold=aug_per_fold, metric=metric, standard_deviation=std_noaug)
                                                
plot_p_values(p_values=p_values)